In [2]:
import sys
sys.path.append('../src/')

from ase.db import connect
import pyro
from models.bnn import BNN
from models.nets.network import NetAtom
import torch
import pytorch_lightning as L
import torch.utils.data as data
import numpy as np
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

from optuna import Study
import optuna
from optuna.trial import Trial

from datamodule.aenet_datamodule import AenetDataModule

/home/riccardo/anaconda3/envs/bayesian/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
datamodele = AenetDataModule('/home/riccardo/bin/repos/aenet-bayesian/examples/liquid_water/train.in')

In [4]:
net = NetAtom(
    datamodele.input_size, 
    datamodele.hidden_size, 
    datamodele.species, 
    datamodele.active_names, 
    datamodele.alpha,
    'cpu'
)

In [5]:
model_kwargs = {'net': net,
        'lr': 0.00055,
        'pretrain_epochs': 0,
        'mc_samples_train': 1,
        'mc_samples_eval': 20, 
        'dataset_size': datamodele.train_size, 
        'fit_context': 'lrt', 
        'prior_loc': 1, 
        'prior_scale': 1, 
        'guide': 'normal', 
        'q_scale': 1.,
        'obs_scale' : 10.}

In [6]:
from pyro.optim import ClippedAdam

ClippedAdam({'lr': 0.0001, 'betas': [0.95, 0.999], 'clip_norm': 15})

In [7]:
early_stopping = EarlyStopping(monitor='elbo/val', min_delta = 0., # minimum change in the monitored quantity to qualify as an improvement
  patience= 3, # number of checks with no improvement after which training will be stopped
  verbose= False, # verbosity mode
  mode= "min", # "max" means higher metric value is better, can be also "min"
  strict= True, # whether to crash the training if monitor is not found in the validation metrics
  check_finite= True,) # when set True, stops training when the monitor becomes NaN or infinite
#   'stopping_threshold'= null, # stop training immediately once the monitored quantity reaches this threshold
#   'divergence_threshold'= null, # stop training as soon as the monitored quantity becomes worse than this threshold
#   'check_on_train_epoch_end'= null,)

early_stopping = EarlyStopping(
    monitor='elbo/val',
    patience=100,
    mode='min' )

In [ ]:
default_root_dir = '../examples/liquid_water/'

def objective(trial: Trial, model_kwargs : dict, output_dir: str):
    # model_kwargs['pretrain_epochs'] = trial.suggest_categorical(
    #         "pretrain_epochs", [0, 5]
    #     )
    model_kwargs['lr'] = trial.suggest_float("lr", 1e-5, 1e-3)
    #model_kwargs['prior_loc'] = trial.suggest_float("prior_loc", 0., 1., log=True)
    model_kwargs['prior_scale'] = trial.suggest_float("prior_scale", 0.1, 5, log=True)
    model_kwargs['q_scale'] = trial.suggest_float("q_scale", 1e-4, 5, log=True)
    model_kwargs['obs_scale'] = trial.suggest_float("obs_scale", 0.1, 5, log=True)

    model = BNN(**model_kwargs)
    trainer = L.Trainer(min_epochs = 1, max_epochs = 20, default_root_dir=default_root_dir, callbacks=early_stopping)
    trainer.fit(model=model, datamodule=datamodele)

    return trainer.callback_metrics['elbo/val'].item()

In [23]:
from sqlalchemy import create_engine

engine = create_engine("sqlite://///home/riccardo/bin/repos/aenet-bnn/src/results/bnn_lrt.db", pool_pre_ping=True)


In [31]:
study = optuna.create_study(
    study_name='bnn_hyperparams',
    storage="sqlite:///../../bnn_lrt.db",
    directions=["minimize"]
)

study.optimize(
        lambda trial: objective(trial, model_kwargs=model_kwargs, output_dir='optuna_log'),
        n_trials=60,
        timeout=None,
        catch=(RuntimeError,),
    )

AssertionError: 